In [38]:
import pandas as pd
import quandl, math
import numpy as np #Numpy helps use arrays, which python otherwise doesn't have.
from sklearn import preprocessing #Preprocessing scales the features between [-1, 1] for efficiency factors.
from sklearn import cross_validation #Shuffles dataset to remove any bias and also helps in breaking the dataset into training and testing dataset.
from sklearn import svm #CozYolo!
from sklearn.linear_model import LinearRegression

api_key = 'kkvFCdVcnysTGUyMzLy1'

quandl.ApiConfig.api_key = api_key

df = quandl.get('WIKI/GOOGL')
print(df.head(2))
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume',]] #Doing this helps us choose what all columns we wanna keep from all the columns that exist
df['HL_PCT'] = ((df['Adj. High'] - df['Adj. Low']) / df['Adj. Low']) * 100 
df['PCT_change'] = ((df['Adj. Close'] - df['Adj. Open']) / df['Adj. Close']) * 100

df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume',]] #Finally, these are the columns that we choose to keep.

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   

            Adj. Volume  
Date                     
2004-08-19   44659000.0  
2004-08-20   22834300.0  


In [59]:
forecast_col = 'Adj. Close' #This is much like String forecast_col = "Adj. Close" in java.
df.fillna(-9999, inplace = True) #We dont wanna lose data while doing ML, so we fill all the N/A(NaN) places with -9999 which we can use to identify an outlier
forecast_out = int(math.ceil(0.01*len(df))) #Defines by what percentage of the dataframe we wanna shift the below columns.
print(forecast_out)

df['label'] = df[forecast_col].shift(-forecast_out) #Shifts the columns above. (Shift column, not row.) Also, df['label'] copies df['forecast_col'] data shifted up by forecast_out value. df['label'] is kinda our dummy label
df.dropna(inplace = True) #This drops all those rows which have N/A(NaN) in them
df.tail() #Prints from back of dataframe.

34


,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2017-10-19,1001.84,1.004713,-0.290466,1677021.0,1049.38
2017-10-20,1005.07,0.636555,-0.197001,1568454.0,1051.97
2017-10-23,985.54,2.308005,-1.992816,1623580.0,1048.77
2017-10-24,988.49,1.246571,0.201317,1416283.0,1051.39
2017-10-25,991.46,1.708870,0.523470,1368042.0,1057.47


In [62]:
X = np.array(df.drop(['label'], 1)) #Drop label col from df and make array
y = np.array(df['label']) #y is label i.e whole 'label'.

X = preprocessing.scale(X) #Scaling is done

# X = X[:forecast_out + 1] #Get X only until forecast_out ##PS - Don't need to do this coz df contains only data till forecast_out as we shifted the data above and dropped the N/A columns.
# y = np.array(df['label'])

print(len(X), len(y))

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

clf = LinearRegression()
clf.fit(X_train, y_train) #Train the classifier on the training dataset

#'fit' and 'score' are 'analogous' to 'train' and 'test'.

accuracy_clf = clf.score(X_test, y_test) #Test the datset on testing dataset

print(accuracy)

support_vector_regression_classifier = svm.SVR()
svr = support_vector_regression_classifier
svr.fit(X_train, y_train)
accuracy_SVR = svr.score(X_test, y_test)
print(accuracy_SVR) 

#As svr yields less accuracy than regular LinearRegression, we don't use svr here.

3321 3321
0.977460984748078
0.8181411612631375
